# Try different models-compare clustering results over eval. set
TODO: try different eps

In [47]:
# load eval.set (duplicates free)
import json
filename = 'pre-clustered_docs_harder.json'
with open(filename, 'r') as file:
    cdocs = json.load(file)


## Title+abstract models
## TODO: lower case text

In [79]:
# train different models
from gensim.utils import simple_preprocess as sp
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing

unwrapped_docs = [doc for cluster in cdocs for doc in cluster]
# title + abstract models
train_corpus_ta = [ TaggedDocument(sp(doc['title'] + doc['abstract']) ,[i]) for i, doc in enumerate(unwrapped_docs)]
epochs = 45
vec_size = 100
models = [
    # dm = 0, simple SG, simpler model, most of the time efficient and accurate
    Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 100'),
    Doc2Vec(dm=0, vector_size=150, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 150, lower eps'),
    Doc2Vec(dm=0, vector_size=500, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 500'),
    Doc2Vec(dm=0, vector_size=800, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 800'),
    Doc2Vec(dm=0, vector_size=1000, negative=5, hs=0, min_count=2, sample=0, 
            epochs=45, workers=multiprocessing.cpu_count(), comment='dm=0, vec = 1000'),
    # dm = 1, CBOW equivalent 
    Doc2Vec(dm=1, vector_size= 100, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=100, alpha=0.05'),
    Doc2Vec(dm=1, vector_size= 150, min_count=2,epochs = epochs, 
            workers=multiprocessing.cpu_count(), alpha= 0.5, comment='dm=1, vec=150, alpha=0.5, hs=1'),
    Doc2Vec(dm=1, vector_size= 400, window=10, negative=5, hs=0, min_count=2, sample=0, 
            epochs = epochs, workers=multiprocessing.cpu_count(), alpha= 0.05, comment='dm=1, vec=400, alpha=.05')
]

# build our vocabulary of words (all the unique words encountered inside our corpus)
for model in models:
    print(model)
    model.build_vocab(train_corpus_ta)
print("Vocabulary created!")

# train the models on the given data!
counter = 0
for model in models:
    print("Training %s" % model)
    %time model.train(train_corpus_ta, total_examples=len(train_corpus_ta), epochs=model.epochs)
    #model.save(MODEL_NAME+str(counter)+'.model')
    counter = counter + 1
#print("Models Saved")

Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
Doc2Vec("dm=0, vec = 150, lower eps",dbow,d150,n5,mc2,t4)
Doc2Vec("dm=0, vec = 500",dbow,d500,n5,mc2,t4)
Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
Doc2Vec("dm=0, vec = 1000",dbow,d1000,n5,mc2,t4)
Doc2Vec("dm=1, vec=100, alpha=0.05",dm/m,d100,n5,w10,mc2,t4)
Doc2Vec("dm=1, vec=150, alpha=0.5, hs=1",dm/m,d150,n5,w5,mc2,s0.001,t4)
Doc2Vec("dm=1, vec=400, alpha=.05",dm/m,d400,n5,w10,mc2,t4)
Vocabulary created!
Training Doc2Vec("dm=0, vec = 100",dbow,d100,n5,mc2,t4)
CPU times: user 1.8 s, sys: 12 ms, total: 1.81 s
Wall time: 914 ms
Training Doc2Vec("dm=0, vec = 150, lower eps",dbow,d150,n5,mc2,t4)
CPU times: user 2.02 s, sys: 4 ms, total: 2.02 s
Wall time: 998 ms
Training Doc2Vec("dm=0, vec = 500",dbow,d500,n5,mc2,t4)
CPU times: user 3.48 s, sys: 28 ms, total: 3.51 s
Wall time: 1.62 s
Training Doc2Vec("dm=0, vec = 800",dbow,d800,n5,mc2,t4)
CPU times: user 5.23 s, sys: 32 ms, total: 5.26 s
Wall time: 2.36 s
Training Doc2Vec("dm=0, vec = 1000

# Evaluate Clustering

In [90]:
import utils
import my_dbscan
import model_evaluation as me

models_accuracy = []
for k, model in enumerate(models):
    # try different eps for some model
    if k == 1:
        eps = 0.19
        eps_increment = 0.12
    else:
        eps = 0.27
        eps_increment = 0.1
    # get list of document vectors
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = eps, eps_increment = eps_increment, n_iterations = 3, verbose = False)
    # get clusters as list of titles
    titles_clusters = utils.getDocTitleFromUrl(unwrapped_docs, urls_cluster_list)
    #data = utils.plot_clusters(titles_clusters)
    # evaluate clustering
    expected_clusters = []
    for docs in cdocs:
        expected_clusters.append([doc['title'] for doc in docs])
    accuracy = me.compute_clustering_accuracy(titles_clusters, expected_clusters)
    print(accuracy)
    models_accuracy.append(accuracy)
    # last check: we want to penalize models that simply cluster all docs together (that's not a valid result)
    # since the test rules does not prevent this
    print("#Number of clusters found: {0}, against number of pre-computed clusters: {1}#\n".format(
        len(urls_cluster_list), len(cdocs))) 

Results occurences:  [(3, 0), (3, 1), (3, 2), (2, 3), (11, 4), (9, 6), (2, 1), (4, 0), (5, 7), (4, 7)]
Accuracy over each cluster:  [75.0, 100.0, 100.0, 100.0, 61.111111111111114, 100.0, 66.66666666666667, 100.0, 50.0, 66.66666666666667]
81.94444444444443
#Number of clusters found: 11, against number of pre-computed clusters: 10#

Results occurences:  [(2, 0), (3, 1), (3, 2), (2, 3), (9, 4), (7, 6), (1, 13), (3, 0), (4, 7), (4, 10)]
Accuracy over each cluster:  [50.0, 100.0, 100.0, 100.0, 50.0, 77.77777777777777, 0, 75.0, 40.0, 66.66666666666667]
65.94444444444444
#Number of clusters found: 14, against number of pre-computed clusters: 10#

Results occurences:  [(3, 0), (3, 0), (3, 0), (2, 1), (15, 2), (9, 3), (2, 0), (4, 0), (6, 4), (4, 4)]
Accuracy over each cluster:  [75.0, 100.0, 100.0, 100.0, 83.33333333333333, 100.0, 66.66666666666667, 100.0, 60.0, 66.66666666666667]
85.16666666666666
#Number of clusters found: 7, against number of pre-computed clusters: 10#

Results occurences:  

# Visualize results graphically

In [92]:
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
# using my api-key
tls.set_credentials_file(username='D4nt3', api_key='FdMB4O6qCfciGDOnLvdQ')
graph_name = 'model_eval_results'

model_descr = [model.comment for model in models]
# plot test-accuracy results, plus silhoutte scores (times 100, since the value is in between 0-1)
trace0 = go.Bar(
    x = model_descr,
    y = models_accuracy,
    name='Clustering Accuracy',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
trace1 = go.Bar(
    x = model_descr,
    y = [s*100 for s in s_scores],
    name='Silhoutte score',
    marker=dict(
        color='rgb(204,50,100)',
    )
    
)

data = [trace0, trace1]
layout = go.Layout(
    title = 'Clustering Accuracy Results',
    xaxis=dict(
        tickfont=dict(
            size=10,
            color='rgb(107, 107, 107)',
            
        ),
        tickangle = -45
    ),
    yaxis=dict(
        title='Clustering accuracy (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
   
    barmode='group',
    bargap=0.2,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename=graph_name)

# Silhoutte score
Silhouette coefficient combines ideas of both cohesion and separation, 
but for individual points, as well as clusters and 
clusterings; the Silhouette Coefficient is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
The Silhoutte score is merely the average of each silhoutte coefficient, computed over each sample.
It's a measure of the goodness of clustering, by assuming the fact that a cluster X is defined good if both every sample inside it is close to each other, and far from any other relatively-near cluster Y.
This is not necessarily true for every shape of cluster.

    The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.

In [91]:
from sklearn.metrics import silhouette_score
s_scores = []
for model in models:
    # get clusters for each model
    doc_vecs = [model.docvecs[j] for j in range(len(model.docvecs))]
    titles = [doc['title'] for doc in unwrapped_docs]
    urls = [doc['url'] for doc in unwrapped_docs]
    # apply dbscan clustering to these vectors
    urls_cluster_list = my_dbscan.apply_dbscan(doc_vecs = doc_vecs, titles = titles, 
                                               urls = urls, subset_length = len(titles),
                                                 eps = 0.27, eps_increment = 0.1, n_iterations = 3, verbose = False)
    
    # get cluster labels, mantaining original docs ordering
    labels = []
    for doc in unwrapped_docs:
        noise = True
        for i, url_list in enumerate(urls_cluster_list):
            if(doc['url'] in url_list):
                labels.append(i) # keep cluster id
                noise = False
        if noise:
            labels.append(-1)
    # make sure they have the same size
    assert len(labels) == len(doc_vecs)
    ss = silhouette_score(doc_vecs, labels , metric='cosine')
    s_scores.append(ss)
    print(ss)

0.24077336
0.21106187
0.22541517
0.23410366
0.23638208
0.11957765
0.0999626
0.16403265


# Conclusions: 
A really low silhoutte score helps us identify models that tend to have very few cluster, hence not really recognizing differences between docs. This is fundamental, since test rules used so far prevent us from recognizing these kinds of models.
Since silhoutte score is an average of silhoutte coefficients, smaller changes of values here may mean greater differences.